In [1]:
# # import findspark
# # findspark.init()
# import cv2
# import os
# import numpy as np
# import imageio

In [58]:
# from pyspark.sql import SparkSession

In [2]:
# from pyspark.ml.image import ImageSchema

In [3]:
# import pyspark # only run after findspark.init()
# from pyspark.sql import SparkSession

In [82]:
# from pyspark import SparkContext, SparkConf
# #from pyspark.sql import SparkSession
# # conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
# # sc = pyspark.SparkContext(conf=conf)
# spark = SparkSession.builder.config('spark.jars.packages','databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11')
# # import sparkdl

In [86]:
# spark = (SparkSession
#             .builder
#             .config('spark.jars.packages', 'databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11')
#             .config("spark.driver.memory", "8g")
#             .getOrCreate()
# )

In [ ]:
# !pyspark --conf spark.driver.memory=50g --conf spark.executor.pyspark.memory=50g

In [1]:
import os
SUBMIT_ARGS = "--packages databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DL with Spark Deep Cognition").config("spark.driver.memory", "8g").getOrCreate()
sc = spark.sparkContext

Py4JError: org.apache.spark.api.python.PythonUtils.getEncryptionEnabled does not exist in the JVM

In [ ]:
from sparkdl import DeepImageFeaturizer

In [ ]:
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from sparkdl.image import imageIO

# tulips_df = ImageSchema.readImages("flower_photos/tulips").withColumn("label", lit(1))
# daisy_df = imageIO.readImagesWithCustomFn("flower_photos/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
# tulips_train, tulips_test, _ = tulips_df.randomSplit([0.1, 0.05, 0.85])  # use larger training sets (e.g. [0.6, 0.4] for getting more images)
# daisy_train, daisy_test, _ = daisy_df.randomSplit([0.1, 0.05, 0.85])     # use larger training sets (e.g. [0.6, 0.4] for getting more images)
# train_df = tulips_train.unionAll(daisy_train)
# test_df = tulips_test.unionAll(daisy_test)

# # Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# # This ensure that each of the paritions has a small size.
# train_df = train_df.repartition(100)
# test_df = test_df.repartition(100)

In [ ]:
df_images_A = spark.read.format("image").load("C:/Users/doudavi/Desktop/ignore/Sudha/Big_Data/Project/asl-alphabet/asl_alphabet_train/asl_alphabet_train/A/").withColumn("label", lit(1))

In [ ]:
df_images_A.read.format("image").load("C:/Users/doudavi/Desktop/ignore/Sudha/Big_Data/Project/asl-alphabet/asl_alphabet_train/asl_alphabet_train/B/").withColumn("label", lit(2))

In [11]:
# df_images_A.show(11)

In [7]:
img_dict = {}
for i in range(1,30):
    img_dict[i] = 0.8
img_dict

{1: 0.8,
 2: 0.8,
 3: 0.8,
 4: 0.8,
 5: 0.8,
 6: 0.8,
 7: 0.8,
 8: 0.8,
 9: 0.8,
 10: 0.8,
 11: 0.8,
 12: 0.8,
 13: 0.8,
 14: 0.8,
 15: 0.8,
 16: 0.8,
 17: 0.8,
 18: 0.8,
 19: 0.8,
 20: 0.8,
 21: 0.8,
 22: 0.8,
 23: 0.8,
 24: 0.8,
 25: 0.8,
 26: 0.8,
 27: 0.8,
 28: 0.8,
 29: 0.8}

In [12]:
tulips_train, tulips_test, _ = df_images_A.randomSplit([0.1, 0.05, 0.85])  # use larger training sets (e.g. [0.6, 0.4] for getting more images)
daisy_train, daisy_test, _ = df_images_B.randomSplit([0.1, 0.05, 0.85])     # use larger training sets (e.g. [0.6, 0.4] for getting more images)
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer 

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=10, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])

p_model = p.fit(train_df)

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

Test set accuracy = 0.9795918367346939


In [18]:
# import org.apache.spark.ml.image.ImageSchema._
# import java.nio.file.Paths

# val images = readImages("path/to/images")

# images.foreach { rrow =>
#   val row = rrow.getAs[Row](0)
#   val filename = Paths.get(getOrigin(row)).getFileName().toString()
#   val imageData = getData(row)
#   val height = getHeight(row)
#   val width = getWidth(row)

#   println(s"${height}x${width}")
# }

In [19]:
# size_img = 64,64 
# images_for_plot = []
# labels_for_plot = []
# for folder in os.listdir(train_dir):
#     for file in os.listdir(train_dir + '/' + folder):
#         filepath = train_dir + '/' + folder + '/' + file
#         image = cv2.imread(filepath)
# #         print("Image print")
# #         print(image)
#         final_img = cv2.resize(image, size_img)
# #         print("final Image print")
# #         print(final_img)
# #         ary = imageStructToArray(final_img)
# #         final_img = Image.fromarray(obj=ary, mode='RGB')
#         imageio.core.util.Image img = imageio.imread(final_img, as_gray=False, pilmode="RGB")
#         #final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
#         #image_df = spark.read.format("image").load(final_img)
# #       print(final_img)
#         #print(type(imref[0][0]))
#         images_for_plot.append(img)
#         labels_for_plot.append(folder)
#         break
# # return images_for_plot, labels_for_plot
# print(images_for_plot)

In [20]:
# size_img = 64,64 
# images_for_plot = []
# labels_for_plot = []
# for folder in os.listdir(train_dir):
#     for file in os.listdir(train_dir + '/' + folder):
#         filepath = train_dir + '/' + folder + '/' + file
#         image = cv2.imread(filepath)
#         print("Image print")
#         print(image)
#         final_img = cv2.resize(image, size_img)
#         print("final Image print")
#         print(final_img)
#         ary = imageStructToArray(final_img)
#         final_img = Image.fromarray(obj=ary, mode='RGB')
# #         imageio.core.util.Image img = imageio.imread(final_img, as_gray=False, pilmode="RGB")
#         final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
#         image_df = spark.read.format("image").load(final_img)
# #       print(final_img)
#         print(type(imref[0][0]))
#         images_for_plot.append(img)
#         labels_for_plot.append(folder)
#         break
# # return images_for_plot, labels_for_plot
# print(images_for_plot)

In [21]:
# def read_images():
#     size_img = 64,64 
#     images_for_plot = []
#     labels_for_plot = []
#     for folder in os.listdir(dir_path):
#         for file in os.listdir(dir_path + '/' + folder):
#             filepath = dir_path + '/' + folder + '/' + file
#             image = cv2.imread(filepath)
#             final_img = cv2.resize(image, size_img)
#             final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
#             images_for_plot.append(final_img)
#             labels_for_plot.append(folder)
#             break
#     return images_for_plot, labels_for_plot

In [54]:
# from pyspark.sql.types import StringType
# from pyspark.sql.functions import udf, col
# def say_hello(name : str) -> str:
#      return f"Hello {name}"
# assert say_hello("Summer") == "Hello Summer"
# say_hello_udf = udf(lambda name: say_hello(name), StringType())
# df = spark.createDataFrame([("Rick",),("Morty",)], ["name"])

In [ ]:
# label_udf = udf(my_udf, ArrayType(StringType()))

In [14]:
# df.show()

In [15]:
# df.withColumn("greetings", say_hello_udf(col("name"))).show()

In [16]:
# image = cv2.imread('C:/Users/doudavi/Desktop/ignore/Sudha/Big_Data/Project/asl-alphabet/asl_alphabet_train/asl_alphabet_train/A/A1.jpg')

In [17]:
# len(image[0])

In [18]:
# final_img = cv2.resize(image, size_img)

In [19]:
# final_img[0]

In [20]:
# final_img1 = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)

In [21]:
# len(final_img1[0])

In [22]:
# df = spark.sql('''select 'spark' as hello ''')
# df.show()

In [23]:
# import numpy as np # We'll be storing our data as numpy arrays
# import os # For handling directories
# from PIL import Image # For handling the images
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg 

In [24]:
# %conda list|grep "sparkdl"